# MASTER DATA SCIENCE: NUCLIO

# Entregable 2: Supervised Machine Learning

### IMPORTAMOS LIBRERÍAS Y CREAMOS FUNCIONES

In [2]:
import pandas as pd  # Librería para la manipulación y el análisis de datos
# Librería para la manipulación de datos y para la ejecución de operaciones matemáticas
import numpy as np
import matplotlib.pyplot as plt  # Librería para la visualización de datos
import seaborn as sns  # Librería para la visualización de datos
# Librería para crear modelos de ML
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder, OneHotEncoder

pd.set_option("display.max_columns", None)


In [3]:
def eliminar_valores(lista, valores):
    """
    Elimina de 'lista' todos los elementos que estén en 'valores'.
    """
    return [x for x in lista if x not in valores]


def eliminar_columnas(df, columnas):
    return df.drop(columns=columnas, errors='ignore', inplace= True)


def obtener_lista_variables(dataset, target):

    lista_numericas = []
    lista_boolean = []
    lista_categoricas = []

    for i in dataset:
        if (dataset[i].dtype.kind == "f" or dataset[i].dtype.kind == "i") and len(dataset[i].unique()) != 2 and (i not in target):
            lista_numericas.append(i)
        elif (dataset[i].dtype.kind == "f" or dataset[i].dtype.kind == "i") and len(dataset[i].unique()) == 2 and (i not in target):
            lista_boolean.append(i)
        elif (dataset[i].dtype.kind == "O") and i not in target:
            lista_categoricas.append(i)

    return lista_numericas, lista_boolean, lista_categoricas


def umbral_columnas_nulos(dataset, columna, umbral):

  lista_umnbral = []
  porcentaje_nulos_n = (
      dataset[columna].isnull().sum() / dataset.shape[0]) * 100

  for i in range(0, len(porcentaje_nulos_n)):
    if (porcentaje_nulos_n.iloc[i]) > umbral:
      lista_umnbral.append(porcentaje_nulos_n.index[i])
      print(porcentaje_nulos_n.index[i], ":", porcentaje_nulos_n.iloc[i] , "% nulos")


  return lista_umnbral


def listas_balanceadas(df, lista, umbral):
    resultados = []
    for col in lista:
        # Calculamos el porcentaje de cada valor único
        porcentaje = df[col].value_counts(normalize=True) * 100

        # Filtrar valores que superan el umbral
        mayores_80 = porcentaje[porcentaje > umbral]

        if not mayores_80.empty:
            for valor, pct in mayores_80.items():
                print(
                    f"Columna: {col}, Valor: '{valor}', Participación: {round(pct, 2)}%")
                resultados.append((col))

    resultados = list(set(resultados))
    resultados_balencedado = eliminar_valores(lista, resultados)
    return resultados_balencedado, resultados


def segmentacion_categorias_menor_umbral(df, lista, umbral):
    categorias_mayor_umbral = []
    categorias_menor_umbral = []
    for i in lista:
        if len(df[i].unique()) >= umbral:
            categorias_mayor_umbral.append(i)
        else:
            categorias_menor_umbral.append(i)
            print(i, " : ", len(df[i].unique()))
    return categorias_mayor_umbral, categorias_menor_umbral


# Filtrar correlaciones mayores al umbral (excluyendo la diagonal y duplicados)
def corr_umbral(corr, limite):
    umbral = limite/100

    corr_pairs = (
        # Mantener solo la parte superior de la matriz
        corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        # Convierte a Series con pares (col1, col2)
            .stack()
            .reset_index()                                         # Pasar a DataFrame
    )
    corr_pairs.columns = ['Columna1', 'Columna2', 'Correlacion']

    # Filtrar los pares que superen el umbral
    corr_filtrado = corr_pairs[corr_pairs['Correlacion'].abs() > umbral]
    return corr_filtrado


## A) Descripción Inicial

In [4]:
# LEEMOS EL FICHERO
df = pd.read_csv("sample_mmp.csv", low_memory=False)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 84 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   Unnamed: 0                                         500000 non-null  int64  
 1   MachineIdentifier                                  500000 non-null  object 
 2   ProductName                                        500000 non-null  object 
 3   EngineVersion                                      500000 non-null  object 
 4   AppVersion                                         500000 non-null  object 
 5   AvSigVersion                                       500000 non-null  object 
 6   IsBeta                                             500000 non-null  int64  
 7   RtpStateBitfield                                   498168 non-null  float64
 8   IsSxsPassiveMode                                   500000 non-null  int64 

In [6]:
df.head(5)

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,8427007,f1cd864e97bae82bdf96523e1a539121,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1234.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,8,85219.0,NaN,205.0,172,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,1443.0,275891.0,4.0,5.0,2273.0,NaN,953869.0,HDD,952838.0,0,8192.0,AllinOne,23.0,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,27.0,120,FullAuto,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,355.0,19951.0,0,NaN,0.0,0,0,0.0,0.0,11.0,1
1,8829090,fd5ba6f5b75325ec0423a6c67cc75942,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1282.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,129,54198.0,NaN,126.0,124,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2102.0,248850.0,4.0,5.0,2660.0,NaN,476940.0,HDD,457600.0,0,8192.0,Notebook,13.2,1280.0,720.0,Mobile,lion,0.000000e+00,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,18.0,72,FullAuto,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,0.0,486.0,48753.0,0,0.0,0.0,0,0,0.0,1.0,3.0,0
2,2731904,4e628391e7cc7c482fb3286f486dbd25,win8defender,1.1.15100.1,4.9.10586.1106,1.273.781.0,0,7.0,0,NaN,46781.0,2.0,1.0,1,149,122689.0,27.0,181.0,56,windows10,x64,10.0.0.0,10586,768,th2,10586.1176.amd64fre.th2_release_sec.170913-1848,Home,1.0,0,NaN,0.0,74.0,RequireAdmin,1.0,1.0,Convertible,Windows.Desktop,2206.0,238234.0,4.0,5.0,2998.0,NaN,244198.0,SSD,221251.0,0,8192.0,Notebook,15.5,1920.0,1080.0,Mobile,lion,0.000000e+00,10.0.10586.1176,amd64,th2_release_sec,10586,1176,Core,CORE,Update,24.0,109,Notify,0,IS_GENUINE,Retail,NaN,0.0,Retail,0.0,554.0,33111.0,1,0.0,0.0,1,1,0.0,0.0,15.0,0
3,1359513,270a2e9d028144a4df12a9e3da79fba5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1639.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,164,114587.0,27.0,205.0,172,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,525.0,265410.0,8.0,5.0,2880.0,NaN,228936.0,SSD,228321.0,0,8192.0,Notebook,15.5,1920.0,1080.0,Mobil

In [7]:
df.describe()

,Unnamed: 0,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,5.000000e+05,500000.000000,498168.000000,500000.000000,24061.000000,498062.000000,498062.000000,498062.000000,500000.000000,500000.000000,481760.000000,345437.000000,499984.000000,500000.000000,500000.000000,500000.000000,498074.000000,500000.000000,470152.000000,496791.000000,494838.000000,4.993770e+05,494619.000000,494236.000000,497653.000000,497653.000000,497651.000000,4.970240e+05,4.970240e+05,500000.000000,495444.000000,497346.000000,497350.000000,497350.000000,4.849620e+05,500000.000000,500000.000000,496668.000000,500000.000000,500000.000000,84775.000000,491067.000000,181896.000000,489651.000000,490939.000000,500000.000000,182334.0,499099.000000,500000.000000,500000.000000,495960.000000,483050.000000,483050.000000,500000.000000
mean,4.458888e+06,0.000002,6.846207,0.017242,1652.824529,47850.908413,1.326763,1.020714,0.987816,108.037454,81271.646027,24.869936,169.730423,122.610998,15726.933842,574.718574,0.945789,0.000028,0.000438,126.662566,0.978241,1.372503e+01,2218.646481,239128.046882,3.994074,4.529069,2370.986509,5.140433e+05,3.780546e+05,0.077034,6129.232176,16.689837,1548.304317,898.239345,1.125600e+09,15841.372572,967.224762,14.605509,60.446306,0.000522,0.000024,0.000014,0.000258,402.675943,33030.988944,0.485438,0.0,0.007067,0.125434,0.037744,0.057245,0.284060,7.888577,0.499906
std,2.575619e+06,0.001414,1.023049,0.130172,1004.754276,14023.085346,0.523000,0.166608,0.109707,63.068540,48985.129677,5.613712,89.325172,69.303165,2188.646069,248.203346,0.226433,0.005291,0.020928,42.634892,0.145895,8.995848e+03,1315.709604,72048.883577,2.071281,1.289635,842.136675,3.704468e+05,3.384722e+05,0.266646,4964.521253,5.932014,368.565945,214.862450,1.888768e+09,1959.440301,2920.627724,10.200946,45.000423,0.022841,0.004857,0.003776,0.016072,221.527124,21220.161428,0.499788,0.0,0.083766,0.331211,0.190577,0.232310,0.450966,4.551764,0.500000
min,2.000000e+00,0.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.000000,0.000000,1.000000,7.000000,1.000000,1.000000,1.000000,7600.000000,16.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000e+00,54.000000,1.000000,1.000000,1.000000,3.000000,1.024000e+04,9.689000e+03,0.000000,512.000000,4.900000,-1.000000,-1.000000,0.000000e+00,9600.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,11.000000,10.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.227692e+06,0.000000,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36825.000000,18.000000,89.000000,74.000000,15063.000000,256.000000,1.000000,0.000000,0.000000,111.000000,1.000000,1.000000e+00,1443.000000,189641.750000,2.000000,5.000000,1998.000000,2.393720e+05,1.207750e+05,0.000000,4096.00

In [8]:
# Identifcador de maquina no me sirve para analisis
eliminar = ['MachineIdentifier', 'Unnamed: 0']
eliminar_columnas(df, eliminar)


In [9]:
df["HasDetections"].isnull().sum()
# Identificar si tengo null en el target


0

 - Cantidad de Duplicados

In [10]:
df[df.duplicated(keep=False)]
# sin duplicados

,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
4337,win8defender,1.1.15200.1,4.18.1807.18075,1.275.11.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,120,120697.0,11.0,144.0,75,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,84790.0,4.0,5.0,2703.0,NaN,488386.0,SSD,487109.0,0,8192.0,Notebook,13.9,1920.0,1080.0,Mobile,NaN,1.000000e+00,10.0.17134.228,amd64,rs4_release,17134,228,ProfessionalEducation,PROFESSIONAL,Upgrade,8.0,31,FullAuto,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,57723.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
15396,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1494.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,41,130775.0,18.0,52.0,217,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,AllInOne,Windows.Desktop,2668.0,24299.0,4.0,5.0,2710.0,NaN,953869.0,HDD,952592.0,0,4096.0,AllinOne,23.0,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,9.0,34,FullAuto,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,628.0,44624.0,1,NaN,0.0,0,0,0.0,0.0,10.0,1
15841,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1198.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,160,35251.0,48.0,204.0,170,windows10,x64,10.0.0.0,16299,256,rs3,16299.637.amd64fre.rs3_release_svc.180808-1748,Pro,1.0,0,NaN,0.0,117.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2102.0,241060.0,4.0,5.0,2421.0,NaN,244198.0,SSD,224776.0,0,8192.0,Desktop,24.1,1920.0,1200.0,Desktop,NaN,4.294967e+09,10.0.16299.665,amd64,rs3_release,16299,665,ProfessionalEducation,PROFESSIONAL,Other,25.0,115,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,486.0,51023.0,1,NaN,0.0,0,0,0.0,1.0,3.0,1
17354,win8defender,1.1.13504.0,4.11.15063.0,1.237.0.0,0,7.0,0,3195.0,53447.0,1.0,1.0,1,57,56452.0,NaN,76.0,222,windows10,x64,10.0.0.0,15063,256,rs2,15063.0.amd64fre.rs2_release.170317-1834,Pro,1.0,0,NaN,NaN,105.0,NaN,1.0,1.0,Desktop,Windows.Desktop,4589.0,313586.0,8.0,1.0,881.0,NaN,476940.0,HDD,200000.0,0,4096.0,Desktop,19.4,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.15063.0,amd64,rs2_release,15063,0,Professional,PROFESSIONAL,IBSClean,9.0,34,UNKNOWN,0,INVALID_LICENSE,Retail,NaN,0.0,Retail,NaN,142.0,9599.0,0,NaN,0.0,0,0,0.0,1.0,10

In [11]:
df2 = df.copy()

In [12]:
df2.drop_duplicates(inplace=True)
# eliminar duplicados


### B) Analisis de Variables Boleanas , Númericas y Categoricas

In [13]:
target = ["HasDetections"]

In [14]:
lista_numericas, lista_boolean, lista_categoricas = obtener_lista_variables(
    df2, target)


In [15]:
print("Cantidad de columnas Totales ", len(df2.columns))
print("Cantidad de columnas númericas " , len(lista_numericas))
print("Cantidad de columnas boleanas ", len(lista_boolean))
print("Cantidad de columnas categoicas ", len(lista_categoricas))
print("Las variables target es:", target[0])


Cantidad de columnas Totales  82
Cantidad de columnas númericas  42
Cantidad de columnas boleanas  10
Cantidad de columnas categoicas  29
Las variables target es: HasDetections


 #### - Analisis de las variables Númericas

In [16]:
df2[lista_numericas].head()

,RtpStateBitfield,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,7.0,NaN,53447.0,1.0,1.0,8,85219.0,NaN,205.0,172,17134,256,1.0,0.0,137.0,1.0,1.0,1443.0,275891.0,4.0,5.0,2273.0,953869.0,952838.0,8192.0,23.0,1920.0,1080.0,4.294967e+09,17134,165,27.0,120,NaN,0.0,NaN,355.0,19951.0,0.0,0.0,0.0,11.0
1,7.0,NaN,53447.0,1.0,1.0,129,54198.0,NaN,126.0,124,17134,256,1.0,0.0,137.0,1.0,1.0,2102.0,248850.0,4.0,5.0,2660.0,476940.0,457600.0,8192.0,13.2,1280.0,720.0,0.000000e+00,17134,165,18.0,72,NaN,0.0,0.0,486.0,48753.0,0.0,0.0,1.0,3.0
2,7.0,NaN,46781.0,2.0,1.0,149,122689.0,27.0,181.0,56,10586,768,1.0,0.0,74.0,1.0,1.0,2206.0,238234.0,4.0,5.0,2998.0,244198.0,221251.0,8192.0,15.5,1920.0,1080.0,0.000000e+00,10586,1176,24.0,109,NaN,0.0,0.0,554.0,33111.0,0.0,0.0,0.0,15.0
3,7.0,NaN,53447.0,1.0,1.0,164,114587.0,27.0,205.0,172,17134,768,1.0,0.0,137.0,1.0,1.0,525.0,265410.0,8.0,5.0,2880.0,228936.0,228321.0,8192.0,15.5,1920.0,1080.0,5.770000e+02,17134,286,27.0,120,NaN,0.0,NaN,142.0,48473.0,0.0,0.0,0.0,15.0
4,7.0,NaN,47238.0,2.0,1.0,207,72421.0,27.0,277.0,75,17134,256,1.0,0.0,137.0,1.0,1.0,1443.0,256478.0,4.0,1.0,198.0,476940.0,464374.0,4096.0,15.5,1366.0,768.0,0.000000e+00,17134,228,8.0,31,NaN,0.0,NaN,355.0,4343.0,0.0,0.0,1.0,13.0


In [17]:
nulos_numericos = umbral_columnas_nulos(df2, lista_numericas , 80)


DefaultBrowsersIdentifier : 95.18845356865694 % nulos
Census_IsFlightingInternal : 83.04377915209895 % nulos


In [18]:
eliminar_columnas(df2, nulos_numericos)
lista_numericas = eliminar_valores(lista_numericas, nulos_numericos)


In [19]:
connulos = umbral_columnas_nulos(df2 ,lista_numericas , 0 )

RtpStateBitfield : 0.36642638269955435 % nulos
AVProductStatesIdentifier : 0.3876279092094631 % nulos
AVProductsInstalled : 0.3876279092094631 % nulos
AVProductsEnabled : 0.3876279092094631 % nulos
CityIdentifier : 3.647662631709483 % nulos
OrganizationIdentifier : 30.91162563704587 % nulos
GeoNameIdentifier : 0.0032002304165899944 % nulos
IsProtected : 0.38522773639702057 % nulos
SMode : 5.968829755742414 % nulos
IeVerIdentifier : 0.6418462129273308 % nulos
Firewall : 1.032474338152347 % nulos
UacLuaenable : 0.12460897184597292 % nulos
Census_OEMNameIdentifier : 1.0762774919794225 % nulos
Census_OEMModelIdentifier : 1.1528830075765455 % nulos
Census_ProcessorCoreCount : 0.4692337848325079 % nulos
Census_ProcessorManufacturerIdentifier : 0.4692337848325079 % nulos
Census_ProcessorModelIdentifier : 0.4696338136345817 % nulos
Census_PrimaryDiskTotalCapacity : 0.5950428430847021 % nulos
Census_SystemVolumeTotalCapacity : 0.5950428430847021 % nulos
Census_TotalPhysicalRAM : 0.9108655823219

In [20]:
df2[connulos].describe()


,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,IsProtected,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSInstallLanguageIdentifier,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
count,498132.000000,498026.000000,498026.000000,498026.000000,481727.000000,345417.000000,499948.000000,498038.000000,470122.000000,496755.000000,494802.000000,4.993410e+05,494583.000000,494200.000000,497618.000000,497618.000000,497616.000000,4.969890e+05,4.969890e+05,495410.000000,497311.000000,497315.000000,497315.000000,4.849270e+05,496632.000000,491032.000000,181895.000000,489616.000000,490904.000000,499063.000000,495925.000000,483017.000000,483017.000000
mean,6.846195,47850.560541,1.326784,1.020716,81271.066276,24.869879,169.731020,0.945787,0.000438,126.662926,0.978242,1.372595e+01,2218.609340,239130.641963,3.994034,4.529044,2370.970361,5.140460e+05,3.780514e+05,6129.160885,16.689700,1548.297900,898.235456,1.125451e+09,14.605436,0.000014,0.000258,402.678127,33031.425391,0.007065,0.057247,0.284062,7.888637
std,1.023085,14023.426963,0.523011,0.166614,48985.630400,5.613194,89.325934,0.226437,0.020928,42.636134,0.145893,8.996172e+03,1315.714492,72045.809119,2.071276,1.289665,842.154420,3.704454e+05,3.384684e+05,4964.475670,5.932102,368.568377,214.865149,1.888688e+09,10.200954,0.003776,0.016073,221.527339,21220.270308,0.083758,0.232313,0.450967,4.551759
min,0.000000,3.000000,1.000000,0.000000,7.000000,1.000000,1.000000,0.000000,0.000000,8.000000,0.000000,0.000000e+00,54.000000,1.000000,1.000000,1.000000,3.000000,1.024000e+04,9.689000e+03,512.000000,4.900000,-1.000000,-1.000000,0.000000e+00,1.000000,0.000000,0.000000,11.000000,10.000000,0.000000,0.000000,0.000000,1.000000
25%,7.000000,49480.000000,1.000000,1.000000,36825.000000,18.000000,89.000000,1.000000,0.000000,111.000000,1.000000,1.000000e+00,1443.000000,189642.000000,2.000000,5.000000,1998.000000,2.393720e+05,1.207750e+05,4096.000000,13.900000,1366.000000,768.000000,0.000000e+00,8.000000,0.000000,0.000000,142.000000,13156.000000,0.000000,0.000000,0.000000,3.000000
50%,7.000000,53447.000000,1.000000,1.000000,82373.000000,27.000000,181.000000,1.000000,0.000000,135.000000,1.000000,1.000000e+00,2102.000000,247520.000000,4.000000,5.000000,2500.000000,4.769400e+05,2.494500e+05,4096.000000,15.500000,1366.000000,768.000000,0.000000e+00,9.000000,0.000000,0.000000,500.000000,33070.000000,0.000000,0.000000,0.000000,10.000000
75%,7.000000,53447.000000,2.000000,1.000000,123938.000000,27.000000,267.000000,1.000000,0.000000,137.000000,1.000000,1.000000e+00,2668.000000,304438.000000,4.000000,5.000000,2877.000000,9.538690e+05,4.759810e+05,8192.000000,17.200000,1920.000000,1080.000000,4.294967e+09,20.000000,0.000000,0.000000,556.000000,52436.000000,0.000000,0.000000,1.000000,11.000000
max,35.000000,70492.000000,5.000000,4.000000,167958.000000,52.000000,296.000000,1.000000,1.000000,429.000000,1.000000,6.357062e+06,6143.000000,345493.000000,88.000000,10.000000,4472.000000,4.768767e+07,4.768710e+07,393216.000000,142.000000,11520.000000,4320.000000,4.294967e+09,39.000000,1.000000,1.000000,1084.000000,72091.000000,1.000000,1.000000,1.000000,15.000000


In [21]:
corr = df2[lista_numericas].corr(numeric_only=True)
corr_umbral(corr ,80)


,Columna1,Columna2,Correlacion
333,OsBuild,Census_OSBuildNumber,0.937658
674,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,0.898249
734,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,0.988489


- La variable "duration" tiene valores negativos.

- La variable "imdb score" tiene valores negativos.

- La variable "title_year" tiene un valor mínimo anormalmente bajo.

MIRA

In [22]:
# Eliminar alta correlacion en numericas 
elimino_col = ['Census_OSBuildNumber', 'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_OSUILocaleIdentifier']

eliminar_columnas(df2, elimino_col)
lista_numericas = eliminar_valores(lista_numericas, elimino_col)


 #### - Analisis de las variables Boleanas

In [23]:
df[lista_boolean].head()


,IsBeta,IsSxsPassiveMode,HasTpm,AutoSampleOptIn,Census_HasOpticalDiskDrive,Census_IsPortableOperatingSystem,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Census_IsPenCapable
0,0,0,1,0,0,0,0,NaN,0,0
1,0,0,1,0,0,0,0,0.0,0,0
2,0,0,1,0,0,0,1,0.0,1,1
3,0,0,1,0,0,0,1,NaN,0,0
4,0,0,1,0,0,0,1,NaN,1,0


In [24]:
nulos_boleanos = umbral_columnas_nulos(df2, lista_boolean, 0)


Census_IsWIMBootEnabled : 63.530774215743534 % nulos


In [25]:
df['Census_IsWIMBootEnabled'].value_counts()


Census_IsWIMBootEnabled
0.0    182334
Name: count, dtype: int64

In [26]:
# es mejor eliminar columna Census_IsWIMBootEnabled :

eliminar_columnas(df2, nulos_boleanos)
lista_boolean = eliminar_valores(lista_boolean, nulos_boleanos)


In [27]:
print(lista_boolean)


['IsBeta', 'IsSxsPassiveMode', 'HasTpm', 'AutoSampleOptIn', 'Census_HasOpticalDiskDrive', 'Census_IsPortableOperatingSystem', 'Census_IsSecureBootEnabled', 'Census_IsTouchEnabled', 'Census_IsPenCapable']


In [28]:
df2[lista_boolean].head(5)

,IsBeta,IsSxsPassiveMode,HasTpm,AutoSampleOptIn,Census_HasOpticalDiskDrive,Census_IsPortableOperatingSystem,Census_IsSecureBootEnabled,Census_IsTouchEnabled,Census_IsPenCapable
0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,1,1,1
3,0,0,1,0,0,0,1,0,0
4,0,0,1,0,0,0,1,1,0


In [29]:
balanceados_boleanos , no_balanceados_boleanos = listas_balanceadas(df2,lista_boolean , 90)

Columna: IsBeta, Valor: '0', Participación: 100.0%
Columna: IsSxsPassiveMode, Valor: '0', Participación: 98.28%
Columna: HasTpm, Valor: '1', Participación: 98.78%
Columna: AutoSampleOptIn, Valor: '0', Participación: 100.0%
Columna: Census_HasOpticalDiskDrive, Valor: '0', Participación: 92.3%
Columna: Census_IsPortableOperatingSystem, Valor: '0', Participación: 99.95%
Columna: Census_IsPenCapable, Valor: '0', Participación: 96.23%


In [30]:
print("Columnas mas balanceadas : " ,balanceados_boleanos)
print("Columnas no balanceadas : " ,no_balanceados_boleanos)


Columnas mas balanceadas :  ['Census_IsSecureBootEnabled', 'Census_IsTouchEnabled']
Columnas no balanceadas :  ['Census_IsPenCapable', 'AutoSampleOptIn', 'Census_IsPortableOperatingSystem', 'IsBeta', 'HasTpm', 'IsSxsPassiveMode', 'Census_HasOpticalDiskDrive']


In [31]:
listas_balanceadas(df2,balanceados_boleanos , 0)

Columna: Census_IsSecureBootEnabled, Valor: '0', Participación: 51.46%
Columna: Census_IsSecureBootEnabled, Valor: '1', Participación: 48.54%
Columna: Census_IsTouchEnabled, Valor: '0', Participación: 87.46%
Columna: Census_IsTouchEnabled, Valor: '1', Participación: 12.54%


([], ['Census_IsSecureBootEnabled', 'Census_IsTouchEnabled'])

In [32]:
eliminar_columnas(df2, no_balanceados_boleanos)
lista_boolean =eliminar_valores(lista_boolean ,no_balanceados_boleanos )

In [33]:
print(lista_boolean)

['Census_IsSecureBootEnabled', 'Census_IsTouchEnabled']


Categoricos

In [34]:
df[lista_categoricas].head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,OsBuildLab,SkuEdition,PuaMode,SmartScreen,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
0,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1234.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,RequireAdmin,Desktop,Windows.Desktop,NaN,HDD,AllinOne,Desktop,NaN,10.0.17134.165,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,OEM:DM,Retail
1,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1282.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,RequireAdmin,Notebook,Windows.Desktop,NaN,HDD,Notebook,Mobile,lion,10.0.17134.165,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,OEM:DM,Retail
2,win8defender,1.1.15100.1,4.9.10586.1106,1.273.781.0,windows10,x64,10.0.0.0,th2,10586.1176.amd64fre.th2_release_sec.170913-1848,Home,NaN,RequireAdmin,Convertible,Windows.Desktop,NaN,SSD,Notebook,Mobile,lion,10.0.10586.1176,amd64,th2_release_sec,Core,CORE,Update,Notify,IS_GENUINE,Retail,Retail
3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1639.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,NaN,NaN,Notebook,Windows.Desktop,NaN,SSD,Notebook,Mobile,NaN,10.0.17134.286,amd64,rs4_release,Core,CORE,IBSClean,FullAuto,IS_GENUINE,Retail,Retail
4,win8defender,1.1.15200.1,4.18.1807.18075,1.275.511.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,NaN,RequireAdmin,Notebook,Windows.Desktop,NaN,HDD,Portable,Mobile,NaN,10.0.17134.228,amd64,rs4_release,Professional,PROFESSIONAL,Update,FullAuto,IS_GENUINE,OEM:DM,Retail


In [35]:
nulos_categoricas =umbral_columnas_nulos(df2,lista_categoricas , 80)

PuaMode : 99.97479818546935 % nulos
Census_ProcessorClass : 99.58357001704123 % nulos


In [36]:
eliminar_columnas(df2 , nulos_categoricas)
lista_categoricas = eliminar_valores(lista_categoricas , nulos_categoricas)

In [37]:
balanceados_categoricos ,  no_balanceados_categoricos = listas_balanceadas(df, lista_categoricas, 90)


Columna: ProductName, Valor: 'win8defender', Participación: 98.92%
Columna: Platform, Valor: 'windows10', Participación: 96.61%
Columna: Processor, Valor: 'x64', Participación: 90.88%
Columna: OsVer, Valor: '10.0.0.0', Participación: 96.77%
Columna: Census_DeviceFamily, Valor: 'Windows.Desktop', Participación: 99.84%
Columna: Census_OSArchitecture, Valor: 'amd64', Participación: 90.89%
Columna: Census_FlightRing, Valor: 'Retail', Participación: 93.66%


In [38]:
eliminar_columnas(df2, no_balanceados_categoricos)
lista_categoricas = eliminar_valores(
    lista_categoricas, no_balanceados_categoricos)


In [39]:
print(lista_categoricas)


['EngineVersion', 'AppVersion', 'AvSigVersion', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'SmartScreen', 'Census_MDC2FormFactor', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSVersion', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel']


In [40]:
categorias_uniq_mayor , categorias_uniq_menor = segmentacion_categorias_menor_umbral(df2 ,lista_categoricas , 10 )

OsPlatformSubRelease  :  9
SkuEdition  :  8
Census_PrimaryDiskTypeName  :  5
Census_OSInstallTypeName  :  9
Census_OSWUAutoUpdateOptionsName  :  6
Census_GenuineStateName  :  4
Census_ActivationChannel  :  6


In [41]:
print("columnas con mas de 10 valores unicos: ",  categorias_uniq_mayor)
print("\n columnas con menos de 10 valores unicos: ", categorias_uniq_menor)


columnas con mas de 10 valores unicos:  ['EngineVersion', 'AppVersion', 'AvSigVersion', 'OsBuildLab', 'SmartScreen', 'Census_MDC2FormFactor', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSVersion', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName']

 columnas con menos de 10 valores unicos:  ['OsPlatformSubRelease', 'SkuEdition', 'Census_PrimaryDiskTypeName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel']


In [42]:
# Copia antes de transformacion de categoricas 
df3 = df2.copy()

In [43]:
df3.head()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSBranch,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1.1.15100.1,4.18.1807.18075,1.273.1234.0,7.0,53447.0,1.0,1.0,8,85219.0,NaN,205.0,172,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,1443.0,275891.0,4.0,5.0,2273.0,953869.0,HDD,952838.0,8192.0,AllinOne,23.0,1920.0,Desktop,NaN,4.294967e+09,10.0.17134.165,rs4_release,165,Professional,PROFESSIONAL,UUPUpgrade,27.0,FullAuto,IS_GENUINE,OEM:DM,0.0,NaN,355.0,19951.0,0,0.0,0,0.0,0.0,11.0,1
1,1.1.15100.1,4.18.1807.18075,1.273.1282.0,7.0,53447.0,1.0,1.0,129,54198.0,NaN,126.0,124,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,RequireAdmin,1.0,1.0,Notebook,2102.0,248850.0,4.0,5.0,2660.0,476940.0,HDD,457600.0,8192.0,Notebook,13.2,1280.0,Mobile,lion,0.000000e+00,10.0.17134.165,rs4_release,165,Professional,PROFESSIONAL,UUPUpgrade,18.0,FullAuto,IS_GENUINE,OEM:DM,0.0,0.0,486.0,48753.0,0,0.0,0,0.0,1.0,3.0,0
2,1.1.15100.1,4.9.10586.1106,1.273.781.0,7.0,46781.0,2.0,1.0,149,122689.0,27.0,181.0,56,10586,768,th2,10586.1176.amd64fre.th2_release_sec.170913-1848,Home,1.0,0.0,74.0,RequireAdmin,1.0,1.0,Convertible,2206.0,238234.0,4.0,5.0,2998.0,244198.0,SSD,221251.0,8192.0,Notebook,15.5,1920.0,Mobile,lion,0.000000e+00,10.0.10586.1176,th2_release_sec,1176,Core,CORE,Update,24.0,Notify,IS_GENUINE,Retail,0.0,0.0,554.0,33111.0,1,0.0,1,0.0,0.0,15.0,0
3,1.1.15200.1,4.18.1807.18075,1.275.1639.0,7.0,53447.0,1.0,1.0,164,114587.0,27.0,205.0,172,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0.0,137.0,NaN,1.0,1.0,Notebook,525.0,265410.0,8.0,5.0,2880.0,228936.0,SSD,228321.0,8192.0,Notebook,15.5,1920.0,Mobile,NaN,5.770000e+02,10.0.17134.286,rs4_release,286,Core,CORE,IBSClean,27.0,FullAuto,IS_GENUINE,Retail,0.0,NaN,142.0,48473.0,1,0.0,0,0.0,0.0,15.0,1
4,1.1.15200.1,4.18.1807.18075,1.275.511.0,7.0,47238.0,2.0,1.0,207,72421.0,27.0,277.0,75,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0.0,137.0,RequireAdmin,1.0,1.0,Notebook,1443.0,256478.0,4.0,1.0,198.0,476940.0,HDD,464374.0,4096.0,Portable,15.5,1366.0,Mobile,NaN,0.000000e+00,10.0.17134.228,rs4_release,228,Professional,PROFESSIONAL,Update,8.0,FullAuto,IS_GENUINE,OEM:DM,0.0,NaN,355.0,4343.0,1,0.0,1,0.0,1.0,13.0,1


### C) Analisis de Variables vs Target

In [44]:
df[target].mean()


HasDetections    0.499906
dtype: float64

In [45]:
print(type(df[target]))

<class 'pandas.core.frame.DataFrame'>


In [46]:
df3[lista_numericas].describe()

,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalBatteryNumberOfCharges,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsVirtualDevice,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
count,498132.000000,498026.000000,498026.000000,498026.000000,499964.000000,481727.000000,345417.000000,499948.000000,499964.000000,499964.000000,499964.000000,498038.000000,470122.000000,496755.000000,494802.000000,4.993410e+05,494583.000000,494200.000000,497618.000000,497618.000000,497616.000000,4.969890e+05,4.969890e+05,495410.000000,497311.000000,497315.000000,4.849270e+05,499964.000000,496632.000000,491032.000000,181895.000000,489616.000000,490904.000000,499063.000000,495925.000000,483017.000000,483017.000000
mean,6.846195,47850.560541,1.326784,1.020716,108.037067,81271.066276,24.869879,169.731020,122.609896,15726.900769,574.736403,0.945787,0.000438,126.662926,0.978242,1.372595e+01,2218.609340,239130.641963,3.994034,4.529044,2370.970361,5.140460e+05,3.780514e+05,6129.160885,16.689700,1548.297900,1.125451e+09,967.237329,14.605436,0.000014,0.000258,402.678127,33031.425391,0.007065,0.057247,0.284062,7.888637
std,1.023085,14023.426963,0.523011,0.166614,63.069010,48985.630400,5.613194,89.325934,69.303037,2188.691152,248.198841,0.226437,0.020928,42.636134,0.145893,8.996172e+03,1315.714492,72045.809119,2.071276,1.289665,842.154420,3.704454e+05,3.384684e+05,4964.475670,5.932102,368.568377,1.888688e+09,2920.633879,10.200954,0.003776,0.016073,221.527339,21220.270308,0.083758,0.232313,0.450967,4.551759
min,0.000000,3.000000,1.000000,0.000000,1.000000,7.000000,1.000000,1.000000,1.000000,7600.000000,16.000000,0.000000,0.000000,8.000000,0.000000,0.000000e+00,54.000000,1.000000,1.000000,1.000000,3.000000,1.024000e+04,9.689000e+03,512.000000,4.900000,-1.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,11.000000,10.000000,0.000000,0.000000,0.000000,1.000000
25%,7.000000,49480.000000,1.000000,1.000000,51.000000,36825.000000,18.000000,89.000000,74.000000,15063.000000,256.000000,1.000000,0.000000,111.000000,1.000000,1.000000e+00,1443.000000,189642.000000,2.000000,5.000000,1998.000000,2.393720e+05,1.207750e+05,4096.000000,13.900000,1366.000000,0.000000e+00,165.000000,8.000000,0.000000,0.000000,142.000000,13156.000000,0.000000,0.000000,0.000000,3.000000
50%,7.000000,53447.000000,1.000000,1.000000,97.000000,82373.000000,27.000000,181.000000,88.000000,16299.000000,768.000000,1.000000,0.000000,135.000000,1.000000,1.000000e+00,2102.000000,247520.000000,4.000000,5.000000,2500.000000,4.769400e+05,2.494500e+05,4096.000000,15.500000,1366.000000,0.000000e+00,285.000000,9.000000,0.000000,0.000000,500.000000,33070.000000,0.000000,0.000000,0.000000,10.000000
75%,7.000000,53447.000000,2.000000,1.000000,162.000000,123938.000000,27.000000,267.000000,182.000000,17134.000000,768.000000,1.000000,0.000000,137.000000,1.000000,1.000000e+00,2668.000000,304438.000000,4.000000,5.000000,2877.000000,9.538690e+05,4.759810e+05,8192.000000,17.200000,1920.000000,4.294967e+09,547.000000,20.000000,0.000000,0.000000,556.000000,52436.000000,0.000000,0.000000,1.000000,11.000000
max,35.000000,70492.000000,5.000000,4.000000,222.000000,167958.000000,52.000000,296.000000,283.000000,18242.000000,784.000000,1.000000,1.000000,429.000000,1.000000,6.357062e+06,6143.000000,345493.000000,88.000

In [47]:
corr = df3.corr(numeric_only=True)
corr['HasDetections'].to_frame().style.background_gradient(cmap='coolwarm')
# corr.style.background_gradient(cmap='coolwarm')


,HasDetections
RtpStateBitfield,0.040312
AVProductStatesIdentifier,0.116182
AVProductsInstalled,-0.148793
AVProductsEnabled,-0.041950
CountryIdentifier,0.006653
CityIdentifier,-0.004530
OrganizationIdentifier,0.005415
GeoNameIdentifier,0.003414
LocaleEnglishNameIdentifier,0.017020
OsBuild,0.024420


In [48]:
df3[lista_categoricas].select_dtypes(include=['object']).describe().T


,count,unique,top,freq
EngineVersion,499964,53,1.1.15200.1,216473
AppVersion,499964,95,4.18.1807.18075,288789
AvSigVersion,499964,6455,1.273.1420.0,5771
OsPlatformSubRelease,499964,9,rs4,220761
OsBuildLab,499963,453,17134.1.amd64fre.rs4_release.180410-1804,206419
SkuEdition,499964,8,Home,308562
SmartScreen,321398,12,RequireAdmin,241591
Census_MDC2FormFactor,499964,12,Notebook,320940
Census_PrimaryDiskTypeName,499255,4,HDD,325410
Census_ChassisTypeName,499927,34,Notebook,294223


In [49]:
df3[lista_boolean].astype(str).select_dtypes(include=['object']).describe()


,Census_IsSecureBootEnabled,Census_IsTouchEnabled
count,499964,499964
unique,2,2
top,0,0
freq,257262,437248


In [50]:
(df3[lista_boolean].sum()/df3.shape[0])*100


Census_IsSecureBootEnabled    48.543895
Census_IsTouchEnabled         12.544103
dtype: float64

In [51]:
# # tranformaciones finales 
# # numericas 
# MinMax = MinMaxScaler()
# df_copy[lista_numericas] = MinMax.fit_transform(df_copy[lista_numericas])



In [52]:
# # Categoricas label encode ( asigna un numero aleatorio a la categoria): a debatir , lo usare para las categorias con muchas variables unicas
# le = LabelEncoder()

# for col in categorias_mayor_10:
#     df_copy[col] = le.fit_transform(df_copy[col])




In [53]:
# # Categoricas one hot encode  ): a debatir , menos variables unicas


# # Transformamos las variables categóricas mediante OHE usando get_dummies
# df_copy_ohe = pd.get_dummies(
#     data=df_copy, columns=categorias_menor_10, dtype=int)



In [54]:
# corr = df_copy_ohe.corr()
# # 

In [55]:
# corr.style.background_gradient(cmap='coolwarm')


In [56]:
# from sklearn.model_selection import train_test_split

# from sklearn.ensemble import RandomForestClassifier


In [57]:
# target

In [58]:
# X_train, X_test, Y_train, Y_test = train_test_split(df_copy_ohe.drop('HasDetections', axis=1),
#                                                     df_copy_ohe['HasDetections'], test_size=0.2)


In [59]:
# rf = RandomForestClassifier(
#     n_estimators=100,  max_depth=5,  random_state=42)

# rf.fit(X_train, Y_train)


In [60]:
# y_pred = rf.predict(X_test)


In [61]:
# from sklearn.metrics import make_scorer
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score
# print("\tAccuracy: %1.3f\n" % accuracy_score(Y_test, y_pred))
# print("\tPrecision: %1.3f" % precision_score(Y_test, y_pred))
# print("\tRecall: %1.3f" % recall_score(Y_test, y_pred))
# print("\tF1: %1.3f\n" % f1_score(Y_test, y_pred))
